<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import pandas as pd
import datetime
import sys

wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'

def get_stock_data(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_excel(f'{raw_stock_data_path}/{date}-{ticker}.xlsx', index=False)
		print(f'Got {raw_stock_data_path}/{date}-{ticker}.xlsx')
	except KeyError:
		print('Wait a little longer')
		sys.exit()

def read_wanted_sheet():
	df = pd.read_excel(wanted_sheet_path)
	for index, row in df.iterrows():
		files = [file for file in os.listdir(raw_stock_data_path)]
		if not (f'{row["Date"].date()}-{row["Ticker"]}.xlsx') in files:
			get_stock_data(row['Date'].date(), row['Ticker'].upper())

read_wanted_sheet()

In [ ]:
import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata'):
	df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])

print(cum_stock_data)
cum_stock_data.to_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx', index=False)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [105]:
import requests
import os
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx')
ddf = pd.DataFrame()

ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
ddf['FTime'] = ddf['FTime'].dt.date

# df.loc[192]
# Get Open
t_df = df.loc[df['FTime'].dt.time==datetime.time(9,30), ['Ticker', 'FTime', 'Open']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Close
t_df = df.loc[df['FTime'].dt.time==datetime.time(15,59), ['Ticker', 'FTime', 'Close']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['High'].transform(max) == t_df['High']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Low
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'Low']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['Low'].transform(min) == t_df['Low']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<=datetime.time(20,0))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df = t_df.groupby(['Ticker', 'FTime'])['Volume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Premarket Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'Volume': 'PVolume'}, inplace=True)
t_df = t_df.groupby(['Ticker', 'FTime'])['PVolume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Gap %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.rename(columns={'FTime': 'Date'}, inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['Gap %'] = (ddf['Open'] / ddf.groupby('Ticker').shift(1)['Close'] - 1) * 100

# Get Volume Ratio
ddf['Vol Ratio'] = ddf['Volume']/ddf['PVolume']

ddf
# print(ddf.head())
# print(ddf.groupby('Ticker')['Close'].shift(1).head())
# t_df

,Ticker,Date,Open,Close,High,Low,Volume,PVolume,Gap %,Vol Ratio
25736,ADN,2022-04-05,2.22,2.0100,2.2800,1.9850,1249714,2330,NaN,536.357940
25082,ADN,2022-04-06,2.58,3.6350,3.9800,2.4700,162103715,1049987,28.358209,154.386402
24268,ADN,2022-04-07,3.32,3.3000,3.4500,3.0400,19160827,2447205,-8.665750,7.829678
23635,ADN,2022-04-08,3.36,2.9401,3.5500,2.8000,9323953,404272,1.818182,23.063564
23131,ADN,2022-04-11,2.81,2.6800,2.8300,2.5700,2888275,131866,-4.425020,21.903106
22558,AXSM,2022-04-19,40.26,40.9200,41.6100,38.0500,11045166,2037628,NaN,5.420600
21843,CASA,2022-04-18,6.04,7.1100,7.1800,5.6300,185110834,15728315,NaN,11.769273
21058,CASA,2022-04-19,6.58,6.5400,6.9500,6.0200,37292653,4309616,-7.454290,8.653359
20870,CELZ,2022-03-22,1.70,1.7000,1.8000,1.6300,469405,4660,NaN,100.730687
20194,CELZ,2022-03-23,3.45,3.1100,3.9000,2.6300,140002440,21615566,102.941176,6.476927
